In [8]:
import pandas as pd 
import numpy as np 

metadata = pd.read_csv("../Project-Disney-World-DSF/metadata.csv")
print(metadata)

hauntedhouse = pd.read_csv("../Project-Disney-World-DSF/haunted_mansion[87].csv")
print(hauntedhouse)


            DATE WDW_TICKET_SEASON  DAYOFWEEK  DAYOFYEAR  WEEKOFYEAR  \
0     01/01/2015               NaN          5          0           0   
1     01/02/2015               NaN          6          1           0   
2     01/03/2015               NaN          7          2           0   
3     01/04/2015               NaN          1          3           1   
4     01/05/2015               NaN          2          4           1   
...          ...               ...        ...        ...         ...   
2074  08/27/2021               NaN          6        238          34   
2075  08/28/2021               NaN          7        239          34   
2076  08/29/2021               NaN          1        240          35   
2077  08/30/2021               NaN          2        241          35   
2078  08/31/2021               NaN          3        242          35   

      MONTHOFYEAR  YEAR          SEASON  HOLIDAYPX  HOLIDAYM  ... HSFIREWKS  \
0               1  2015  CHRISTMAS PEAK          0      

In [38]:
def createList (r1, r2):
    return list(range(r1, r2))

indeces = createList(0, 23) + createList(29, 63) + createList(90, 101) + createList(134, 165)
type(indeces)
metadata1 = metadata.iloc[:, indeces]
metadata1.isna().sum()



IndexError: positional indexers are out-of-bounds